In [249]:
import pandas as pd
from itertools import product

Se usará modelado estrella ya que no se tiene tanta información como para denormalizar las dimensiones.

Una dimension. Tabla dim_user_plans

Una tabla de hechos tabla fct_sales (guarda id_usuario,  numero de mensajes, duracion total de llamadas, uso total de internet, date), relaciona las tablas de internet, calls y messages en una sola tabla de gastos del usuario por cada fecha (periodo).

In [250]:
#Cargar todos los datasets de clean
megaline_calls = pd.read_csv('../data/clean/megaline_calls.csv')
megaline_internet = pd.read_csv('../data/clean/megaline_internet.csv')
megaline_messages = pd.read_csv('../data/clean/megaline_messages.csv')
megaline_plans = pd.read_csv('../data/clean/megaline_plans.csv')
megaline_users = pd.read_csv('../data/clean/megaline_users.csv')

In [251]:
#Crear tabla dim_user_plans con un merge de megaline_users y megaline_plans
#a través de la columna plan de megaline_users y plan_name de megaline_plans
dim_user_plans=pd.merge(megaline_users,megaline_plans,left_on='plan',right_on='plan_name',how='left')#se hace un left join para que se mantengan todos los user_id
#aunque no tengan un plan asociado
#Se eliminan la columna plan ya que son redundantes
dim_user_plans.drop(columns=['plan'],inplace=True)
dim_user_plans.head()

,user_id,first_name,last_name,age,city,reg_date,churn_date,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_mb,usd_per_message,usd_per_minute,plan_name
0,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,2261-12-31,1000,30720,3000,70,0.007,0.01,0.01,ultimate
1,1001,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,2261-12-31,50,15360,500,20,0.010,0.03,0.03,surf
2,1002,Carlee,Hoffman,36,"Las Vegas-Henderson-Paradise, NV MSA",2018-10-21,2261-12-31,50,15360,500,20,0.010,0.03,0.03,surf
3,1003,Reynaldo,Jenkins,52,"Tulsa, OK MSA",2018-01-28,2261-12-31,50,15360,500,20,0.010,0.03,0.03,surf
4,1004,Leonila,Thompson,40,"Seattle-Tacoma-Bellevue, WA MSA",2018-05-23,2261-12-31,50,15360,500,20,0.010,0.03,0.03,surf


In [252]:

#Crear tabla fct_sales
#Se añade la columna user_id, date que recopila todos los user_id y la session_date (internet), call_date (calls) y message_date (messages)
#de los datasets de calls, internet y messages respectivamente sin duplicados. 
#se debe relacionar cada user_id con una fecha en la que uso al menos uno de los servicios.


#Obtener cada user_id de las tablas de calls, internet y messages sin duplicados
usuarios=pd.concat([megaline_calls['user_id'], megaline_internet['user_id'], megaline_messages['user_id']]).drop_duplicates()
#Obtener cada fecha de las tablas de calls, internet y messages sin duplicados
dates=pd.concat([megaline_calls['call_date'], megaline_internet['session_date'], megaline_messages['message_date']]).drop_duplicates()

#Cruzar cada user_id con cada fecha
fct_sales=pd.DataFrame(product(usuarios,dates),columns=['user_id','date'])
print(len(dates))
print(len(fct_sales.loc[fct_sales['user_id']==1000]))


351
351


In [253]:
#Para cada user_id y fecha se obtiene el total de minutos, mensajes y mb consumidos
#importante agrupar por user_id y date en el caso de que exista mas de un consumo de servicios en un mismo dia
fct_sales=pd.merge(fct_sales,megaline_calls.groupby(['user_id','call_date'])['duration'].sum().reset_index(),how='left',left_on=['user_id','date'],right_on=['user_id','call_date'])
fct_sales.rename(columns={'duration':'total_minutes'},inplace=True)
fct_sales=pd.merge(fct_sales,megaline_messages.groupby(['user_id','message_date'])['id'].count().reset_index(),how='left',left_on=['user_id','date'],right_on=['user_id','message_date'])
fct_sales.rename(columns={'id':'total_messages'},inplace=True)
fct_sales=pd.merge(fct_sales,megaline_internet.groupby(['user_id','session_date'])['mb_used'].sum().reset_index(),how='left',left_on=['user_id','date'],right_on=['user_id','session_date'])
fct_sales.rename(columns={'mb_used':'total_mb'},inplace=True)
fct_sales.head()

,user_id,date,call_date,total_minutes,message_date,total_messages,session_date,total_mb
0,1000,2018-12-27,2018-12-27,48.26,2018-12-27,3.0,2018-12-27,880.22
1,1000,2018-12-28,2018-12-28,29.00,2018-12-28,1.0,2018-12-28,660.40
2,1000,2018-12-30,2018-12-30,4.22,2018-12-30,1.0,NaN,NaN
3,1000,2018-12-31,2018-12-31,19.29,2018-12-31,3.0,2018-12-31,0.00
4,1000,2018-12-26,2018-12-26,14.36,2018-12-26,1.0,2018-12-26,270.99


In [254]:
#Quitar las columnas que no se necesitan
fct_sales.drop(columns=['call_date','message_date','session_date'],inplace=True)




In [255]:
#Ver si hay nulos o duplicados en las tabla de fct_sales
print("Tabla fct_sales, nulos")
print(fct_sales.isnull().sum())
print("\nTabla fct_sales, duplicados")
print(fct_sales.duplicated().sum())

Tabla fct_sales, nulos
user_id                0
date                   0
total_minutes     120031
total_messages    138747
total_mb          122957
dtype: int64

Tabla fct_sales, duplicados
0


In [256]:
#A los nulos de total_minutes, total_messages y total_mb se les asigna 0
fct_sales['total_minutes'].fillna(0, inplace=True)
fct_sales['total_messages'].fillna(0, inplace=True)
fct_sales['total_mb'].fillna(0, inplace=True)
print("\nTabla fct_sales, nulos")
print(fct_sales.isnull().sum())


Tabla fct_sales, nulos
user_id           0
date              0
total_minutes     0
total_messages    0
total_mb          0
dtype: int64


In [257]:
#Se pasan las nuevas tablas a data_clean
dim_user_plans.to_csv('../data/clean/dim_user_plans.csv', index=False)
fct_sales.to_csv('../data/clean/fct_sales.csv', index=False)